[View in Colaboratory](https://colab.research.google.com/github/KristianHiggins/fastaic1v3/blob/master/fastaic1v3e1.ipynb)

<h1>FastAI</h1>
<h2>Course 1. Version 3.</h2>
<a href="https://www.youtube.com/watch?v=7hX8yKCX6xM">Episode 1</a>
